# Task 1 - Teoría

1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

2. Defina en sus propias palabras el término “Arc Consistency”

# Task 2 - CSP con Backtracking, Beam y Local Search

## Definición de variables y restricciones

In [3]:
from itertools import combinations

# Definimos los dias disponibles para los examenes
dias = ['lunes', 'martes', 'miercoles']

# Definimos los exámenes y los estudiantes que toman cada examen
examenes = ['IA', 'ADA', 'COMPIS', 'SO', 'BD2', 'MD']
estudiantes_por_examen = {
    'IA': ['Angel', 'Alejandro'],
    'ADA': ['Angel', 'Diego'],
    'COMPIS': ['Alejandro', 'Mark'],
    'SO': ['Angel', 'Mark'],
    'BD2': ['Alejandro', 'Diego'],
    'MD': ['Diego', 'Mark'],
}


# Definimos las variables que representan los exámenes y sus posibles días
variables = {examen: dias for examen in examenes}


## Verificar si la asignación es válida

In [4]:
# Definimos la función para verificar si una asignación es válida
def esValida(asignacion):
    # Verificar que no haya dos examenes en el mismo dia
    for dia in dias:
        examenesPorDia = [examen for examen, diaAsignado in asignacion.items() if diaAsignado == dia]
        if len(examenesPorDia) > 1:
            return False
    
    # Verificar que los estudiantes que toman el mismo curso no tengan examen el mismo dia
    for examen1, examen2 in combinations(examenes, 2):
        if asignacion[examen1] == asignacion[examen2]:
            estudiantes_examen1 = set(estudiantes_por_examen[examen1])
            estudiantes_examen2 = set(estudiantes_por_examen[examen2])
            if len(estudiantes_examen1.intersection(estudiantes_examen2)) > 0:
                return False
    
    return True